# The battle of the neighbourhoods
## Installing and importing everything necessary

In [1]:
#folder: https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DS0701EN/DS0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb?lti=true
!pip install lxml
#!pip install geopy
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
print('###INSTALLATION FINISHED###')

     |████████████████████████████████| 5.5MB 832kB/s eta 0:00:01     |▊                               | 133kB 5.4MB/s eta 0:00:01     |██▍                             | 409kB 5.4MB/s eta 0:00:01     |███▉                            | 655kB 5.4MB/s eta 0:00:01     |█████████▏                      | 1.6MB 5.4MB/s eta 0:00:01     |██████████████████              | 3.1MB 5.4MB/s eta 0:00:01��███▉      | 4.5MB 5.4MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------

In [15]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
import csv
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import folium # map rendering library#
from folium.features import DivIcon
from pprint import pprint

from pathlib import Path#for checking if file exists

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## Defining the necessary functions and global variables

In [3]:
CLIENT_ID = 'CXP5UVSMV52KBO13AECGEOLCZO4B1HQKJLNBEQAIG4Q0QT3A' # my Foursquare ID
CLIENT_SECRET = 'T2MPSQTD4UWQ51EZXVOL0C0FVNMN3JIYFPOTGPO5MVBG3HNX' # my Foursquare Secret. Will reset it, recommend for you, the reviewer, to do the same
VERSION = '20180605' # Foursquare API version
LIMIT = 100 #Seems to be 50, not 100
call_count = 0


In [68]:
def get_venues(lat, lng, radius=750): #this method needs the lat/loong of the city center and searches for all venues of the category Bars (Nightlife) in a radius of 750m around that location
     
    venues_list=[]
    # create the API request URL
    '''url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT,
        search)'''
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        '4bf58dd8d48988d116941735', #categoryId for Bars (Nightlife) from https://developer.foursquare.com/docs/build-with-foursquare/categories/
        lat, 
        lng, 
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]["venues"]
    for v in results:
        venues_list.append([
            v['id'],
            v['name'],
            v['location']['lat'],
            v['location']['lng'],
            get_rating(v['id'])])
        
    df_cityvenues = pd.DataFrame(venues_list, columns=['id', 'venue_name', 'latitude', 'longitude','rating'])
    
    return(df_cityvenues)

In [69]:
def get_rating(venueid):#this method takes a Foursquare venueID and returns the rating. There are venues without rating so there exists error-handling
    
    global call_count
    call_count +=1
    if call_count==250:#these calls are limited to 500/day, so put in a warning
        print("Half of the daily calls used!")
    if call_count==450:
        print("LESS THAN 50 CALLS LEFT!") #https://developer.foursquare.com/docs/places-api/rate-limits/
       
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        venueid,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        )
    # make the GET request
    results = requests.get(url).json()#["response"]["venues"]
    if results['meta']['code'] == 429:
        raise Exception("Errorcode 429: Numer of quotas exceeded for today, sorry")

    try:#not every venue has a rating
        return results['response']['venue']['rating']
    except:
        return np.NaN

In [70]:
def get_geoloc(city):#The input of this method is a city name, it returns the latitude and longitude of that city
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]

In [7]:
#credit to: https://stackoverflow.com/questions/50786266/writing-dictionary-of-dataframes-to-file
#these two methods save and load dataframes that are contained within a dictionary. This is the structure of data here: a dictionary where the key is the city with values as dataframes of venues
def saver(dictex):
    for key, val in dictex.items():
        val.to_csv("data_{}.csv".format(str(key)))

    with open("keys.txt", "w") as f: #saving keys to file
        f.write(str(list(dictex.keys())))

def loader():
    """Reading data from keys"""
    with open("keys.txt", "r") as f:
        keys = eval(f.read())

    dictex = {}    
    for key in keys:
        dictex[key] = pd.read_csv("data_{}.csv".format(str(key)))
        dictex[key].drop(columns=['Unnamed: 0'], inplace= True)

    return dictex

# Starting by initializing a dictionary conatining the Dataframes of the cities with the right header

In [71]:
#initialize list of cities
citylist = ["Freiburg im Breisgau","Berlin","Munich","Toronto","Manhattan"]#list of cities to search for venues
citydict = dict()#dict to contain the city as key and the Dataframe with the venues as value for simple accessing
cityloc = dict()#dict to contain the lat/long of each city
cityrating = dict()#empty dict for the rating of cities

df_template = pd.DataFrame(columns=['id', 'venue_name', 'latitude', 'longitude','rating'])#template of df to initialize correct headers

for i in citylist:#initialize both dicts
    citydict[i] = df_template
    cityloc[i] = get_geoloc(i)
    cityrating[i] = 0

print(cityloc)

{'Freiburg im Breisgau': [47.9960901, 7.8494005], 'Berlin': [52.5170365, 13.3888599], 'Munich': [48.1371079, 11.5753822], 'Toronto': [43.6534817, -79.3839347], 'Manhattan': [40.7896239, -73.9598939]}


### Looking for saved data. If there is any: load that data. If not: make the GET requests to the Forusquare API

In [73]:
keys = Path("keys.txt")
if keys.is_file():
    citydict = loader()
else:
    for i in citylist:
        citydict[i] = citydict[i].append(get_venues(cityloc[i][0], cityloc[i][1]))   #to change to 1000m activate! 1000m uses up the daily quotas with one call!
        citydict[i].sort_values(by=['rating'], ascending=False)
    saver(citydict)
    print("DID A CALL TO FOURSQUARE; USED UP SOME OF THE DAILY CALLS")    

In [74]:
for i in citylist:#sort the venues by rating in the dataframes
    citydict[i].sort_values(by=['rating'], ascending=False, inplace=True)

In [75]:
for i in citylist:#initialize both dicts
    cityrating[i] = citydict[i].head(n=10)['rating'].sum()
    print("The rating of "+i+" is "+ str(cityrating[i]))

max_value = max(cityrating.values())  #finding out which city has the best rating
best_cities = [k for k, v in cityrating.items() if v == max_value] # getting all keys containing the `maximum`

print(max_value, best_cities)
latitude = cityloc[best_cities[0]][0]
longitude =  cityloc[best_cities[0]][1]
pubcrawl = citydict[best_cities[0]].head(n=10)
pubcrawl.head()

The rating of Freiburg im Breisgau is 80.5
The rating of Berlin is 81.89999999999999
The rating of Munich is 86.5
The rating of Toronto is 81.00000000000001
The rating of Manhattan is 34.800000000000004
86.5 ['Munich']


,id,venue_name,latitude,longitude,rating
14,572f9097498ed768a7f4cca2,The High,48.133101,11.572939,9.2
3,4bbc6329afe1b7136d4d304b,Biergarten am Viktualienmarkt,48.135194,11.576365,8.8
7,4cec06cb595cb1f7f16bdc14,Weinhaus Neuner,48.137694,11.568340,8.8
17,529cad3411d216970da64d58,Déjà Bu?,48.133001,11.576824,8.8
5,4ade0ca8f964a5205b6821e3,Bar Centrale,48.136821,11.579873,8.7


# Showing a map of the ten best venues

In [76]:
# create the map for the winning city
map_pubcrawl = folium.Map(location=[latitude, longitude], zoom_start=17)

for lat, lon, name in zip(pubcrawl['latitude'], pubcrawl['longitude'], pubcrawl['venue_name']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(#marker for the pubs
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=0.7).add_to(map_pubcrawl)
  
    folium.Marker(#label for the pubs
        [lat, lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<b><div style="font-size: 16pt">{}</div></b>'.format(name.replace("'", "")))
    ).add_to(map_pubcrawl)

    
folium.CircleMarker(#marker for city center
        [cityloc[best_cities[0]][0], cityloc[best_cities[0]][1]],
        radius=5,
        popup='City center',
        color='red',
        fill=True,
        fill_opacity=0.7).add_to(map_pubcrawl)

map_pubcrawl

In [ ]:
'''from openrouteservice import distance_matrix

pubs_coords = [feat['geometry']['coordinates'] for feat in pubs_smoker]

request = {'locations': pubs_coords,
           'profile': 'driving-car',
           'metrics': ['duration']}
    
pubs_matrix = clnt.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(pubs_matrix['durations']),len(pubs_matrix['durations'][0])))


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])

tsp_size = len(pubs_addresses)
num_routes = 1
start = 0 # arbitrary start location
coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    dist_callback = getDistance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
#         while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(pubs_coords[routing.IndexToNode(index)])
            route += str(pubs_addresses[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(pubs_addresses[routing.IndexToNode(index)])
        optimal_coords.append(pubs_coords[routing.IndexToNode(index)])
        print("Route:\n" + route)'''

In [ ]:
#parsed = get_venues(cityloc["Berlin"][0], cityloc["Berlin"][1], radius=1000)
#pprint(parsed)
#print(get_venues(cityloc[i][0], cityloc[i][1], radius=1000))

#OLD STUFF BUT WORKED
#for i in citylist:
    #citydict[i] = citydict[i].append(get_venues(cityloc[i][0], cityloc[i][1]))   #to change to 1000m activate!
    
# tips for route planning https://openrouteservice.org/example-optimize-pub-crawl-with-ors/

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Namensnennung - Nicht-kommerziell - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>. Markus Gern - 04.02.2021